In [2]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_history_aware_retriever

from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_community.chat_models import ChatOllama
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.output_parsers import StrOutputParser

from dotenv import load_dotenv
from langchain_teddynote import logging
from operator import itemgetter


# .env 환경 변수 로딩
# ----------------
load_dotenv()

# LangSmith를 이용하여 LLM 추적
# -------------------------
logging.langsmith("PROD_RAG", set_enable=True)  # enable


model_name_path = "../../HUGGING_FACE_MODEL/BAAI_bge-m3"
def embeddings_call():
    return HuggingFaceEmbeddings(
        model_name=model_name_path,
        model_kwargs={"device": "mps"},  # cpu : 'cpu', macOS: 'mps', CUDA: 'cuda'
        # 모델이 CPU에서 실행되도록 설정. GPU를 사용할 수 있는 환경이라면 'cuda'로 설정할 수도 있음
        encode_kwargs={
            "normalize_embeddings": True
        },  # 임베딩 정규화. 모든 벡터가 같은 범위의 값을 갖도록 함. 유사도 계산 시 일관성을 높여줌
        # cache_folder='../embedding/model',
    )

# vectordb = FAISS.load_local(persist_directory, embedding, index_name, allow_dangerous_deserialization=True)
embeddings = embeddings_call()
vectordb = Chroma(
            persist_directory="../../Chroma_DB/chroma_bank_law_db",
            embedding_function=embeddings,
            collection_name="bank_law_case",
        )

retriever = vectordb.as_retriever(
    search_type="similarity", search_kwargs={"k": 1}
)


llm = ChatOllama(model="EEVE-Korean-10.8B:latest")

contextualize_q_system_prompt = """Given a chat history and the latest user question \
which might reference context in the chat history, formulate a standalone question \
which can be understood without the chat history. Do NOT answer the question, \
just reformulate it if needed and otherwise return it as is."""

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)


qa_system_prompt = """You are an assistant for question-answering tasks. \
Use the following pieces of retrieved context to answer the question. \
If you don't know the answer, just say that you don't know. \
Use three sentences maximum and keep the answer concise.\
\n\n
{context}"""
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

# question_answer_chain = create_stuff_documents_chain(llm, qa_prompt, output_parser=parser)
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

store = {}
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

LangSmith 추적을 시작합니다.
[프로젝트명]
PROD_RAG


/opt/anaconda3/envs/prod_llm/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [3]:
conversational_rag_chain.invoke(
    # {"input": "What is Task Decomposition?"},
    {"input": "전산운영위원회 위원을 알려줘?"},
    config={
        "configurable": {"session_id": "abc123"}
    },
)["answer"]

'전산운영위원회의 상임위원으로는 다음과 같은 분들이 있습니다: 전산담당임원, 종합기획부장, 인사지원부장, 마케팅기획부장, 준법감시부장, IT기획부장, IT개발부장, 정보보호부장, 디지털플랫폼부장. 위원회의 위원장은 전산담당본부장이 맡으며, 유고 시에는 IT기획부장이 직무대행을 합니다. 또한 심의사항 관련 부서의 담당 본부장과 부·실장으로 구성된 위촉위원도 있습니다.'

In [1]:
while True:
    query = input("전산운영위원회 위원을 알려줘")

    for chunk in conversational_rag_chain.stream(
        {"input": query,},
        config={
            "configurable": {
                "session_id": "demo_1"
            }
        }
    ):
        if answer_chunk := chunk.get("answer"):
            print(f"{answer_chunk}", end="", flush=True)
    
    # print()

In [1]:
# from langchain.chains import create_retrieval_chain
# from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_history_aware_retriever

from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_community.chat_models import ChatOllama
# from langchain_core.chat_history import BaseChatMessageHistory
# from langchain_community.chat_message_histories import ChatMessageHistory
# from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.output_parsers import StrOutputParser

from dotenv import load_dotenv
from langchain_teddynote import logging
from operator import itemgetter


# .env 환경 변수 로딩
# ----------------
load_dotenv()

# LangSmith를 이용하여 LLM 추적
# -------------------------
logging.langsmith("PROD_RAG", set_enable=True)  # enable


model_name_path = "../../HUGGING_FACE_MODEL/BAAI_bge-m3"
def embeddings_call():
    return HuggingFaceEmbeddings(
        model_name=model_name_path,
        model_kwargs={"device": "mps"},  # cpu : 'cpu', macOS: 'mps', CUDA: 'cuda'
        # 모델이 CPU에서 실행되도록 설정. GPU를 사용할 수 있는 환경이라면 'cuda'로 설정할 수도 있음
        encode_kwargs={
            "normalize_embeddings": True
        },  # 임베딩 정규화. 모든 벡터가 같은 범위의 값을 갖도록 함. 유사도 계산 시 일관성을 높여줌
        # cache_folder='../embedding/model',
    )

# vectordb = FAISS.load_local(persist_directory, embedding, index_name, allow_dangerous_deserialization=True)
embeddings = embeddings_call()
vectordb = Chroma(
            persist_directory="../../Chroma_DB/chroma_bank_law_db",
            embedding_function=embeddings,
            collection_name="bank_law_case",
        )

retriever = vectordb.as_retriever(
    search_type="similarity", search_kwargs={"k": 1}
)


llm = ChatOllama(model="EEVE-Korean-10.8B:latest")

'''
contextualize_q_system_prompt = """Given a chat history and the latest user question \
which might reference context in the chat history, formulate a standalone question \
which can be understood without the chat history. Do NOT answer the question, \
just reformulate it if needed and otherwise return it as is."""

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        ("human", "{input}"),
    ]
)

history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)
'''

qa_system_prompt = """You are an assistant for question-answering tasks. \
Use the following pieces of retrieved context to answer the question. \
If you don't know the answer, just say that you don't know. \
Use three sentences maximum and keep the answer concise.\
\n\n
{context}"""
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        ("human", "{input}"),
    ]
)

chain = (
    {
        "context": itemgetter("input") | retriever,
        "input": itemgetter("input"),
    }
    | qa_prompt
    | llm
    | StrOutputParser()
)
# # question_answer_chain = create_stuff_documents_chain(llm, qa_prompt, output_parser=parser)
# question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

# rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)


# store = {}
# def get_session_history(session_id: str) -> BaseChatMessageHistory:
#     if session_id not in store:
#         store[session_id] = ChatMessageHistory()
#     return store[session_id]

# conversational_rag_chain = RunnableWithMessageHistory(
#     rag_chain,
#     get_session_history,
#     input_messages_key="input",
#     history_messages_key="chat_history",
#     output_messages_key="answer",
# )

LangSmith 추적을 시작합니다.
[프로젝트명]
PROD_RAG


/opt/anaconda3/envs/prod_llm/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [12]:
chain.invoke(
    {"input": "전산운영위원회 위원을 알려줘?"},
)

'해당 문서에 따르면, 전북은행의 전산운영위원회 상임위원은 다음과 같습니다:\n\n1. 전산담당임원\n2. 종합기획부장\n3. 인사지원부장\n4. 마케팅기획부장\n5. 준법감시부장\n6. IT기획부장\n7. IT개발부장\n8. 정보보호부장\n9. 디지털플랫폼부장 (2022년 8월 8일 개정)\n\n위촉위원은 위원회의 심사에 관련된 부서의 담당 본부장과 부·실장으로 구성되며, 위원회 위원장이 선임합니다.'

In [3]:
from langchain_teddynote.messages import stream_response

answer = chain.stream(
    {"input": "전산운영위원회 위원을 알려줘?"},
)
stream_response(answer)

해당 문서에 따르면, 전북은행의 전산운영위원회 상임위원은 다음과 같습니다:

1. 전산담당임원
2. 종합기획부장
3. 인사지원부장
4. 마케팅기획부장
5. 준법감시부장
6. IT기획부장
7. IT개발부장
8. 정보보호부장
9. 디지털플랫폼부장 (2022년 8월 8일 개정)

위촉위원은 위원회의 심사에 관련된 부서의 담당 본부장과 부·실장으로 구성되며, 위원회 위원장이 선임합니다.

from langchain_teddynote.messages import stream_response
stream_response(answer)

In [13]:
chain.invoke(
    {"input": "전산운영위원회 위원을 알려줘?"},
)

'해당 문서에 따르면, 전북은행의 전산운영위원회 상임위원은 다음과 같습니다:\n\n1. 전산담당임원\n2. 종합기획부장\n3. 인사지원부장\n4. 마케팅기획부장\n5. 준법감시부장\n6. IT기획부장\n7. IT개발부장\n8. 정보보호부장\n9. 디지털플랫폼부장 (2022년 8월 8일 개정)\n\n위촉위원은 위원회의 심사에 관련된 부서의 담당 본부장과 부·실장으로 구성되며, 위원회 위원장이 선임합니다.'

In [14]:
response = chain.stream(
    {"input": "전산운영위원회 위원을 알려줘?"},
)
ai_answer = ""
for token in response:
    ai_answer += token
    
